In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

In [3]:
CONN_SCHEMA='sbx_team_digitcamp'

In [4]:
%%time
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    close_reason_type_name,
                    appl_stage_name

             from {}.{} where (inn = '2221191879') and
                              (creation_month in ('2021-02-01', 
                                                  '2021-03-01',
                                                  '2021-04-01',
                                                  '2021-05-01') )'''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

CPU times: user 1.68 ms, sys: 1.56 ms, total: 3.24 ms
Wall time: 3.28 s


In [5]:
%%time
deals.count()

CPU times: user 301 ms, sys: 501 ms, total: 801 ms
Wall time: 39min 41s


3

In [7]:
load_table(SBX_TEAM_DIGITCAMP, 'tmp_visit_part', hive).show()

,visitNumber,cid,commonSegmentoUID,sbbolUserId,hitPagePath,hitPageTitle,sessionStartTime,sessionDate,hitType,hitNumber,hitTime,eventCategory,eventAction,eventLabel,hitPageHostName,ctl_loading,sessionId,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity
0,1248,1452727868.1587947876,None,None,/ic/dcb/,СберБизнес,1628897025,2021-08-14,EVENT,3,0,[std]: login,show,[news]: [show],localhost,11002249,1628897025,False,None,desktop,Khabarovsk Krai,Khabarovsk
1,2,2143600002.1628852233,None,None,/main,СберБанк Онлайн,1628945635,2021-08-14,PAGE,28,100685,None,None,None,web1.online.sberbank.ru,11002249,1628945635,False,None,desktop,Republic of Bashkortostan,Uchaly
2,1,1043955251.1628925029,pMchzHNegren,None,/help/business/sbbol/100296,Как создать платёжное поручение — СберБанк,1628925029,2021-08-14,EVENT,16,163562,SITE_Other_.help.business.sbbol.100296,informing_leads,scroll 50 percent,www.sberbank.ru,11002249,1628925029,False,None,desktop,Krasnodar Krai,Anapa
3,1581,556968546.1388994877,None,ec615264-6a3f-4fa6-8a87-f6e373fe4547,/main,СберБизнес,1628931602,2021-08-14,EVENT,20,26251,[operations]: offers,impression-story,[placement: main[main_feed]][container: a9ba93ee-d5c6-d10d-e054-90e2ba8333f9][bc: adv_non_bank][product: rabota_ru],sbi.sberbank.ru,11002249,1628931602,False,None,desktop,Vologda Oblast,Vologda
4,480,1598249319.1589364527,None,8a3f5f37-6361-4c1f-94fc-2b8f71d6cd03,/main,СберБизнес,1628925210,2021-08-14,EVENT,46,394568,[std]: layout,show,[layout.primarymenusbbol3]: [novelty_info],sbi.sberbank.ru,11002249,1628925210,False,None,desktop,Republic of Bashkortostan,Ufa
5,247,1333304315.1572956738,None,None,/payments/provider#/main,СберБанк Онлайн,1628940116,2021-08-14,PAGE,21,357979,None,None,None,web1.online.sberbank.ru,11002249,1628940116,True,SM-A305FN,mobile,Sakhalin Oblast,Yuzhno-Sakhalinsk
6,3,1061117300.1626119128,fbOtmfdaV4vn,None,/ru/legal/investments/globalmarkets/sbmarkets,Система электронной торговли SberCIB Terminal — СберБанк,1628971805,2021-08-14,PAGE,38,826865,None,None,None,www.sberbank.ru,11002249,1628971805,False,None,desktop,Saint Petersburg,Saint Petersburg
7,636,1107583888.1591630230,None,3c25f07c-09d4-fc84-e054-90e2ba85d0c8,/accounts-payments/account/:accountId#lightbox/payment/:type/:id,СберБизнес,1628945019,2021-08-14,EVENT,60,182900,[field]: payments,ruble_payment_edit__rub_comm,counterpartyaccountnumber,sbi.sberbank.ru,11002249,1628945019,False,None,desktop,Moscow Oblast,Dmitrov
8,22,2142242428.1628617076,None,None,/CSAFront/index.do,Вход - СберБанк Онлайн,1628950212,2021-08-14,PAGE,4,7806,None,None,None,online.sberbank.ru,11002249,1628950212,True,iPhone,mobile,Rostov Oblast,Rostov-on-Don
9,2,28175819.1628341735,None,None,/CSAFront/oidc/authorize.do?response_type=code&scope=openid+addresses+address_reg+birthdate+citizenship+education+email+gender+inn+maindoc+marital_status+mobile+name+openid+place_of_birth+previous_maindoc+previous_name+snils+verified&client_id=0916a770-fa84-464d-8457-f802ba408ef6&state=GINRH4jxUhWg&nonce=1056s0itG9Pm&redirect_uri=https://domclick.ru/cas/login,Авторизация - Сбер ID,1628961733,2021-08-14,EVENT,5,7239,SBOL_WEB_oidc,Вход по QR-коду,Получен QR,online.sberbank.ru,11002249,1628961733,False,None,desktop,Tambov Oblast,Tambov


### Load_data 

### prod_dct

In [122]:
%%time
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(SBX_TEAM_DIGITCAMP, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)").cache()

CPU times: user 2.58 ms, sys: 955 µs, total: 3.53 ms
Wall time: 75.6 ms


In [123]:
%%time
prod_dct.count()

CPU times: user 3.01 ms, sys: 184 µs, total: 3.19 ms
Wall time: 352 ms


131

### nontop

In [5]:
%%time
nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DlfTTM >= timestamp('2021-04-01'))"\
                  .format(SBX_TEAM_DIGITCAMP, 'MA_MMB_OFFER_NONTOP')).cache()

CPU times: user 2.28 ms, sys: 2.46 ms, total: 4.74 ms
Wall time: 426 ms


In [6]:
%%time
nontop.count()

CPU times: user 18.5 ms, sys: 24.9 ms, total: 43.4 ms
Wall time: 1min 57s


34327204

In [ ]:
nontop.show(3)

In [17]:
nontop.dtypes

[('OFFER_ID', 'bigint'),
 ('INN', 'string'),
 ('NONTOP_FLG', 'string'),
 ('PRIORITY_NUM', 'float'),
 ('CRM_ID', 'string'),
 ('ACTIVITY_TYPE_CD', 'string'),
 ('COEF_PRODUCT_CD', 'float'),
 ('NONTOP_RANG', 'bigint'),
 ('CREATE_DTTM', 'timestamp'),
 ('OFFER_DESC', 'string'),
 ('NPV_VOLUME_HEAP', 'float'),
 ('NPV_INCOME_HEAP', 'float'),
 ('SOURCES_LIST', 'string'),
 ('SOURCES_LV2_LIST', 'string'),
 ('NOTPRPR_FLG', 'bigint'),
 ('CAMPAIGN_CODE', 'string'),
 ('REASON', 'string'),
 ('START_DTTM', 'timestamp'),
 ('END_DTTM', 'string'),
 ('CM_CKR_FLG', 'bigint'),
 ('CM_MS_FLG', 'bigint'),
 ('CM_MKK_FLG', 'bigint'),
 ('SBBOL_FLG', 'bigint'),
 ('PRODUCT_ID', 'bigint'),
 ('CM_GKM_FLG', 'bigint')]

есть кейсы, где потенциалы разные по одинаковым продуктам за одну дату (источники могут повторяться, а могут различаться)

То есть, при сборке потенциалов по разным продуктам, необходимо собирать сет по источникам, потенциалы по продукту при этом суммировать

In [ ]:
%%time
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .show(10)

In [ ]:
%%time
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

бывают кейсы, где в таблице разные потенциалы по совпадающим источникам - их несопоставимо мало, не будем обращать на них внимание

In [ ]:
%%time
nontop.groupby("INN", 'PRODUCT_ID', 'SOURCES_LV2_LIST')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .show(10)

In [61]:
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

12676

сколько строк с одним нн с одним продуктом и разными потенциалами

In [62]:
nontop.groupby("INN", 'PRODUCT_ID', 'COEF_PRODUCT_CD')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

5930

In [ ]:
nontop.filter((f.col("INN") == "263408599005") & (f.col("PRODUCT_ID") == 18)).show()

In [ ]:
nontop.filter((f.col("INN") == "7814503581") & (f.col("PRODUCT_ID") == 156)).show()

посмотрим, как выглядят источники в нонтопе

In [32]:
%%time
nontop.select("SOURCES_LV2_LIST").distinct().show(4)

CPU times: user 15.2 ms, sys: 1.32 ms, total: 16.5 ms
Wall time: 4.11 s


,SOURCES_LV2_LIST
0,"OTR2,ЦА-ДМСМ-0121"
1,"T020,ЦА-ПРОД2ГИСММБ-0120,ЦА-ФОТММБ-0221,NBA"
2,ЦА-ЭКВПИРР-0421
3,"T099,ЦА-ДБКМЛ-0421,NBA"


выделим все источники в список

In [29]:
source_list = nontop.select("SOURCES_LV2_LIST").distinct().collect()

In [ ]:
source_list

In [22]:
source_list[0][0].split(',')

['OTR2', 'ЦА-ДМСМ-0121']

Выделим необходимые столбцы

In [82]:
nontop_need_columns = nontop.select("INN",
                                    "PRODUCT_ID",
                                    "SOURCES_LV2_LIST",
                                    f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_NONTOP"),
                                    f.col("COEF_PRODUCT_CD").alias("POTENTIAL")).cache()

In [83]:
%%time
nontop_need_columns.count()

CPU times: user 196 µs, sys: 2.96 ms, total: 3.16 ms
Wall time: 1.44 s


34327204

In [ ]:
nontop_need_columns.show()

In [85]:
nontop_need_columns.groupby("INN", "PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST"))\
                   .filter((f.col("INN") == "7814503581") & (f.col("PRODUCT_ID") == 156)).show()

,INN,PRODUCT_ID,collect_set(SOURCES_LV2_LIST)
0,7814503581,156,"[T145, ЦА-НЕКОМАКТ-1220]"


In [ ]:
nontop_need_columns.select(f.col("SOURCES_LV2_LIST"),
                           f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCE_LST")).distinct().show(100)

### nontop pipeline

таким образом опишем весь пайплайн того, что происходит с нонтопом

In [87]:
nontop_need_columns = nontop.select(f.to_date(f.col('CREATE_DTTM')).alias('CREATE_DATE'),
                                    "INN",
                                    "PRODUCT_ID",
                                    f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_NONTOP"),
                                    f.col("COEF_PRODUCT_CD").alias("POTENTIAL_NONTOP")).cache()

In [88]:
%%time
nontop_need_columns.count()

CPU times: user 2.04 ms, sys: 1.53 ms, total: 3.57 ms
Wall time: 1.67 s


34327204

In [ ]:
nontop_need_columns.show(3)

In [117]:
%%time
nontop_all_sources = nontop_need_columns.groupby("CREATE_DATE",
                                                 "INN",
                                                 "PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST_NONTOP")\
                                                                    .alias("SOURCES_LV2_LIST_NONTOP"),
                                                                   f.avg("POTENTIAL_NONTOP").alias("POTENTIAL_NONTOP")).cache()

CPU times: user 3.13 ms, sys: 3.11 ms, total: 6.24 ms
Wall time: 28.9 ms


In [ ]:
nontop_all_sources.show(5)

In [113]:
def list_to_set(source_lst):
    a = set()
    for i in source_lst:
        a = a | set(i)
    return list(a)      

In [115]:
set_from_list_udf = f.udf(list_to_set, ArrayType(StringType()))

In [119]:
nontop_before_end = nontop_all_sources.withColumn("SOURCES_LV2_LIST_NONTOP",
                                                  set_from_list_udf(f.col("SOURCES_LV2_LIST_NONTOP")))

In [ ]:
%%time
nontop_before_end.show()

In [209]:
nontop_final = nontop_before_end.join(prod_dct, nontop_before_end.PRODUCT_ID == prod_dct.ID, how='inner')\
                                .select("CREATE_DATE",
                                        "INN",
                                        f.col("CRM_PRODUCT_ID").alias("CRM_PRODUCT_ID"),
                                        f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                        f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                        "SOURCES_LV2_LIST_NONTOP",
                                        "POTENTIAL_NONTOP"
                                        ).cache()

In [210]:
nontop_final.count()

33296952

In [ ]:
nontop_final.show()

Оставить в итоге, 


INN	CRM_PRODUCT_ID PRODUCT_GROUP	PRODUCT_SUBGROUP	CREATE_DATE	POTENTIAL, SOURCES_LV2_LIST


список источников, как фичи оставить, словарик с регулярным обновлением, если придут новые источники

In [ ]:
nontop_full_agg.union()

scaller

In [123]:
from pyspark.ml.feature import StandardScaler, VectorAssembler, MinMaxScaler

In [124]:
nontopScaller = MinMaxScaler()
nontopScaller.setInputCol('POTENTIAL_VECTOR')
nontopScaller.setOutputCol('POTENTIAL_SCALED')

MinMaxScaler_1a9364da3769

In [125]:
assembler = VectorAssembler().setInputCols(['POTENTIAL']).setOutputCol('POTENTIAL_VECTOR')

In [ ]:
%%time
assembler.transform(nontop_full_agg).show()

In [127]:
%%time
scaled_nontop_full = nontopScaller.fit(assembler.transform(nontop_full_agg))\
                                  .transform(assembler.transform(nontop_full_agg))

CPU times: user 29.1 ms, sys: 37.3 ms, total: 66.4 ms
Wall time: 3.69 s


In [128]:
unlist = f.udf(lambda x: float(list(x)[0]), DoubleType())

In [ ]:
scaled_nontop_full.show(3)

In [130]:
scaled_nontop_full = scaled_nontop_full.withColumn("POTENTIAL_SCALED",
                                                   unlist("POTENTIAL_SCALED"))

In [ ]:
scaled_nontop_full.show(3)

In [ ]:
max_potential = scaled_nontop_full.select(f.max("POTENTIAL")).take(1)[0][0]
scaled_nontop_full.filter(f.col('POTENTIAL') == max_potential).show()

не очень хорошо, что минимальный потенциал приравнивается к 0, надо сделать его не от 0 до 1 а от epsilon до 1

### OFFER_PRIORITY

In [130]:
priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('2021-04-01')"\
                    .format(SBX_TEAM_DIGITCAMP, 'OFFER_PRIORITY')).cache()

In [132]:
priority.count()

1406268

In [ ]:
%%time
priority.show(3)

In [47]:
print(priority.columns)

['offer_id', 'run_id', 'load_id', 'load_dttm', 'crm_product_id', 'offer_priority', 'user_id', 'update_dttm', 'end_dt', 'activity_type_cd', 'start_dt', 'km_group_id', 'offer_desc', 'sources_list', 'sources_lv2_list', 'campaign_code', 'npv_volume_heap', 'npv_income_heap', 'inn', 'crm_id', 'product_cd', 'reason', 'cm_ckr_flg', 'cm_ms_flg', 'cm_mkk_flg', 'sbbol_flg', 'mkk_activity', 'product_id']


в приорити даты только до 6 мая??

In [51]:
%%time
priority.select('load_dttm').distinct().show(100)

CPU times: user 6 ms, sys: 6.9 ms, total: 12.9 ms
Wall time: 554 ms


,load_dttm
0,2021-04-30
1,2021-04-20
2,2021-04-22
3,2021-04-13
4,2021-04-27
5,2021-05-06
6,2021-04-06
7,2021-04-07
8,2021-04-15


в priority нет даты старта, но есть load_dttm - использую ее, как эквивалентную 

In [10]:
priority_part = priority.select('INN', 
                                'CRM_PRODUCT_ID', 
                                f.col('offer_priority').alias("POTENTIAL"),
                                f.col('load_dttm').alias("CREATE_DTTM")
                                )

In [11]:
priority_part = priority_part.join(product_dict_sp,
                                   on='CRM_PRODUCT_ID',
                                   how='inner').select('INN', 
                                                       'CRM_PRODUCT_ID', 
                                                       'PRODUCT_CD_MMB',
                                                       'PRODUCT_SHORT_NM',
                                                       'PRODUCT_GROUP',
                                                       'PRODUCT_SUBGROUP',
                                                       'POTENTIAL').filter("PRODUCT_CD_MMB is not Null")

In [15]:
%%time
priority_part.count()

CPU times: user 1.33 ms, sys: 1.58 ms, total: 2.91 ms
Wall time: 514 ms


1294404

In [ ]:
%%time
priority_part.show()

### priority pipeline 

In [133]:
priority_need_columns = priority.select(f.to_date(f.col('load_dttm')).alias('CREATE_DATE'),
                                        "INN",
                                        "CRM_PRODUCT_ID",
                                        f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_PRIORITY"),
                                        f.col("offer_priority").alias("POTENTIAL_PRIORITY")).cache()

In [139]:
%%time
priority_all_sources = priority_need_columns.groupby("CREATE_DATE",
                                                     "INN",
                                                     "CRM_PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST_PRIORITY")\
                                                                        .alias("SOURCES_LV2_LIST_PRIORITY"),
                                                                       f.avg("POTENTIAL_PRIORITY")\
                                                                        .alias("POTENTIAL_PRIORITY")).cache()

CPU times: user 2.42 ms, sys: 3.93 ms, total: 6.35 ms
Wall time: 28.3 ms


In [142]:
priority_before_end = priority_all_sources.withColumn("SOURCES_LV2_LIST_PRIORITY",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_PRIORITY")))

In [ ]:
priority_before_end.show()

In [206]:
priority_final = priority_before_end.join(prod_dct, priority_before_end.CRM_PRODUCT_ID == prod_dct.CRM_PRODUCT_ID, how='inner')\
                                .select("CREATE_DATE",
                                        "INN",
                                        priority_before_end.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                        f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                        f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                        "SOURCES_LV2_LIST_PRIORITY",
                                        f.col("POTENTIAL_PRIORITY")
                                        ).cache()

In [207]:
%%time
priority_final = priority_final.filter("CREATE_DATE = '2021-04-30'").cache()

CPU times: user 1.67 ms, sys: 1.65 ms, total: 3.32 ms
Wall time: 15.1 ms


In [208]:
priority_final.count()

311821

In [ ]:
priority_final.show()

In [ ]:
nontop_final.show()

In [177]:
nontop_final.count()

33296952

### toxic 

### join offers 

In [214]:
%%time
result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                             .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                            nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE).alias("CREATE_DATE"),
                                     f.when(f.isnull(priority_final.INN),
                                            nontop_final.INN).otherwise(priority_final.INN).alias("INN"),
                                     f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                            nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID).alias("CRM_PRODUCT_ID"),
                                     f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                            nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP).alias("PRODUCT_GROUP"),
                                     f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                            nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP).alias("PRODUCT_SUBGROUP"),
                                     f.col("SOURCES_LV2_LIST_NONTOP"),
                                     f.col("POTENTIAL_NONTOP"),
                                     f.col("SOURCES_LV2_LIST_PRIORITY"),
                                     f.col("POTENTIAL_PRIORITY"))\
                             .cache()

CPU times: user 5.78 ms, sys: 6.36 ms, total: 12.1 ms
Wall time: 65.5 ms


In [215]:
result_table.count()

33581861

In [ ]:
result_table.show()

сделать юнион источников нонтоп и приорити

### union offers 

In [22]:
mean_priority = priority_part.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                            f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                            f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                            f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                            f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                           ).orderBy(f.col("MEAN_POTENTIAL").desc())

mean_priority.cache()

DataFrame[CRM_PRODUCT_ID: string, MEAN_POTENTIAL: int, PRODUCT_CD_MMB: string, PRODUCT_SHORT_NM: string, PRODUCT_GROUP: string, PRODUCT_SUBGROUP: string]

### union offers 

In [ ]:
union_offers = mean_priority.unionAll(mean_nontop).select('CRM_PRODUCT_ID', 
                                                          'PRODUCT_CD_MMB', 
                                                          'PRODUCT_SHORT_NM',
                                                          'PRODUCT_GROUP',
                                                          'PRODUCT_SUBGROUP',
                                                          'MEAN_POTENTIAL')
union_offers = union_offers.drop_duplicates()